In [ ]:
import ultralytics
from ultralytics import YOLO
import cv2 as cv
from cv2 import aruco
import numpy as np
import math
from datetime import datetime

In [ ]:
import socket
import json

In [ ]:
# deistortion coefficients
dst = np.array([[ 2.32796489e-01, -1.14545687e+00, -5.67938912e-03, -5.39522300e-04, 1.31368375e+00]])
# set coordinate system
#marker_length = 3.85 # unit: cm
marker_length = 3.40 # unit: cm
#marker_length = 5.35 # unit: cm
obj_points = np.array([[-marker_length/2, marker_length/2, 0],
                       [marker_length/2, marker_length/2, 0],
                       [marker_length/2, -marker_length/2, 0],
                       [-marker_length/2, -marker_length/2, 0]], dtype=np.float32).reshape(-1, 1, 3)

In [ ]:
# 'DICT_4X4_50' dictionary: a marker dictionary with 4x4 grid of markers (50 different markers)
marker_dict = aruco.getPredefinedDictionary(aruco.DICT_4X4_50)
# Create detector parameters
detector_params = aruco.DetectorParameters()
# create a detector
detector = aruco.ArucoDetector(marker_dict, detector_params)

In [ ]:
# udp parameters
server_ip = "192.168.1.77"   # ip address (run ipconfig in commend)
port = 8899                  # port number
sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)

In [ ]:
class MyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        else:
            return super(MyEncoder, self).default(obj)

In [ ]:
def rvecEulerAngle(rvec):
    R, _ = cv.Rodrigues(rvec)
    sy = math.sqrt(R[0,0] * R[0,0] +  R[1,0] * R[1,0])
    singular = sy < 1e-6
    if  not singular :
        x = math.atan2(R[2,1] , R[2,2])
        y = math.atan2(-R[2,0], sy)
        z = math.atan2(R[1,0], R[0,0])
    else :
        x = math.atan2(-R[1,2], R[1,1])
        y = math.atan2(-R[2,0], sy)
        z = 0
    return np.array([x, y, z])*(180/np.pi) # covnert radian to degrees


In [ ]:
def find_edge(img):
    gray_img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    gray_img = cv.GaussianBlur(img, (5, 5), 0)
    edged_img = cv.Canny(gray_img, 35, 125)
    # cv.imshow("chessboard", edged_img)
    countours, hierarchy = cv.findContours(edged_img.copy(), cv.RETR_LIST, cv.CHAIN_APPROX_SIMPLE)
    s = max(countours, key=cv.contourArea) 
    ret = cv.minAreaRect(s)
    return ret

In [ ]:
def find_focal_length():
    Xc = 25.55    # unit [cm], object height
    Zc = 52.00      # unit [cm]
    img = cv.imread('.\pic_for_focal_length\calibration.jpg')
    h1, w1 = img.shape[:2]
    newCameraMtx, roi = cv.getOptimalNewCameraMatrix(mtx, dst, (w1, h1), 1, (w1, h1))
    # Undistortion Method 1: using cv.undistort()
    dst1 = cv.undistort(img, mtx, dst, None, newCameraMtx)
    # crop the img
    x, y, w1, h1 = roi
    dst1 = dst1[y:y+h1, x:x+w1]
    # save the result img
    # cv.imwrite('calibration_methond1_result.jpg', dst1)
    edge = find_edge(dst1)
    # edge[1][0] is width，edge[1][1] is height
    focal_length = (edge[1][1] * Zc) / Xc
    # print('focal length is: ', focal_length)
    return focal_length

In [ ]:
focal_length = find_focal_length()

In [ ]:
# Loading a pre-trained YOLOV8n model  
model = YOLO("yolov8n.pt", "v8")

In [ ]:
# open camera
cap = cv.VideoCapture(0)
if cap.isOpened():
    cap.set(cv.CAP_PROP_FRAME_WIDTH, 1920)
    cap.set(cv.CAP_PROP_FRAME_HEIGHT, 1080)

In [ ]:
while cap.isOpened():
    # read one frame, read successfully will return true, otherwise return false
    ret, frame = cap.read()
    if ret:
        h1, w1 = frame.shape[:2]
        newCameraMtx, roi = cv.getOptimalNewCameraMatrix(mtx, dst, (w1, h1), 1, (w1, h1))
        # Undistortion Method 1: using cv.undistort()
        dst1 = cv.undistort(frame, mtx, dst, None, newCameraMtx)
        # crop the img
        x, y, w1, h1 = roi
        dst1 = dst1[y:y+h1, x:x+w1]
        frame = dst1
        detection_output = model.predict(source= frame, conf=0.25, save=False)
        # convert the frame into gray frame
        gray_frame = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
        # find all the aruco markers in the image
        marker_cornes, marker_ids, reject_candidates = detector.detectMarkers(gray_frame)
        frame = detection_output[0].plot()
        # print("----------------Aruco marker detection info--------------:")
        if marker_ids is not None and len(marker_ids) > 0:
            aruco.drawDetectedMarkers(frame, marker_cornes, marker_ids)
            rvecs, tvecs = [], []
            info_dic = {}
            id_list = []
            position_list = []
            for i in range(len(marker_cornes)):
                id_dic = {}
                p_dic = {}
                _, rvec, tvec = cv.solvePnP(obj_points, marker_cornes[i], mtx, dst)
                rvecs.append(rvec)
                tvecs.append(tvec)
                id_dic["id"] = marker_ids[i][0]
                id_list.append(id_dic)
                p_dic["pos1"] = tvec[0][0]
                p_dic["pos2"] = tvec[1][0]
                p_dic["pos3"] = tvec[2][0]
                deg = rvecEulerAngle(rvecs[i])
                p_dic["rota1"] = deg[0]
                p_dic["rota2"] = deg[1]
                p_dic["rota3"] = deg[2]
                position_list.append(p_dic)
                # print(position_list)
            for i in range(len(marker_cornes)):
                cv.drawFrameAxes(frame, mtx, dst, rvecs[i], tvecs[i], marker_length) 
                # deg = rvecEulerAngle(rvecs[i])
                corners = marker_cornes[i].reshape(4, 2)
                corners = corners.astype(int)
                top_left = corners[0].ravel()
                dis = math.sqrt((tvecs[i][0][0] ** 2) + (tvecs[i][1][0] ** 2) + (tvecs[i][2][0] ** 2))
                #cv.putText(frame, 'distance:'+str(round(dis, 2))+str('cm'), top_left, cv.FONT_HERSHEY_PLAIN, 1.3, (255, 0, 255), 2, cv.LINE_AA)
                # print("EulerAngle:", deg)
                # print("Rotation:\n", rvecs[i])
                # print("X-Y-Z Position:\n", tvecs[i])
                #print(tvecs[i][0][0]) 
                #print(tvecs[i][1][0])
                #print(tvecs[i][2][0])
            
            info_dic["markerID"] = id_list
            info_dic["position"] = position_list
            print(info_dic)
            json_message = json.dumps(info_dic, cls=MyEncoder)
            sock.sendto(json_message.encode(), (server_ip, port))

        # else:
        #     cv.putText(frame, "No id", (0,64), cv.FONT_HERSHEY_PLAIN, 1.5, (0, 0, 255),2,cv.LINE_AA)
        
        print("----------------pin-hole camera model info--------------:")
        for r in detection_output:
            for box in r.boxes:
                obj_id = int(box.cls)  # cls is the class value of the boxes
                obj_name = model.names[obj_id]
                print("id: %d, name: %s" % (obj_id, obj_name))
                # xyxy (x_min, y_min, x_max, y_max) represents the coordinates of the corners of the bounding box
                # relative to the top-left corner of the frame or image
                coordinate = box.xyxy[0].cpu().numpy() # cpu(): move the object to CPU memory
                print(coordinate)
                if (obj_name == 'cell phone'):
                    Xc = 13.85 # unit [cm], object height
                    h = max((coordinate[2] - coordinate[0]), (coordinate[3] - coordinate[1]))
                    distance = focal_length * Xc / h
                    print('distance is: %.2f cm'  % distance)
                    #cv.putText(frame, 'distance:'+str(round(distance, 2))+str('cm'), 
                    #          (int(coordinate[0]), int(coordinate[3])), cv.FONT_HERSHEY_PLAIN, 
                    #          1.3, (200, 100, 0), 2, cv.LINE_AA)
                if (obj_name == 'book'):
                    Xc = 24.5 # unit [cm], object height
                    h = max((coordinate[2] - coordinate[0]), (coordinate[3] - coordinate[1]))
                    distance = focal_length * Xc / h
                    print('distance is: %.2f cm'  % distance)
                    #cv.putText(frame, 'distance:'+str(round(distance, 2))+str('cm'), 
                    #          (int(coordinate[0]), int(coordinate[3])), cv.FONT_HERSHEY_PLAIN, 
                    #          1.3, (200, 100, 0), 2, cv.LINE_AA)
                if (obj_name == 'bottle'):
                    Xc = 19.2 # unit [cm], object height
                    h = (coordinate[3] - coordinate[1])
                    distance = focal_length * Xc / h
                    print('distance is: %.2f cm'  % distance)
                    #cv.putText(frame, 'distance:'+str(round(distance, 2))+str('cm'), 
                    #          (int(coordinate[0]), int(coordinate[3])), cv.FONT_HERSHEY_PLAIN, 
                    #          1.3, (200, 100, 0), 2, cv.LINE_AA)
                if (obj_name == 'cup'):
                    Xc = 9.9 # unit [cm], object height
                    h = (coordinate[3] - coordinate[1])
                    distance = focal_length * Xc / h
                    print('distance is: %.2f cm'  % distance)
                    #cv.putText(frame, 'distance:'+str(round(distance, 2))+str('cm'), 
                    #          (int(coordinate[0]), int(coordinate[3])), cv.FONT_HERSHEY_PLAIN, 
                    #          1.3, (200, 100, 0), 2, cv.LINE_AA)

        cv.imshow('camera', frame)
        key = cv.waitKey(10) # 1 means delay in 1 milliseconds
        if key == ord('q'): # input 'q' to break the loop
            break
        elif key == ord('s'):  # input 's' to save the pic
            cv.imwrite(str(datetime.now().strftime("%Y%m%d_%H%M%S")) + ".jpg", frame)

In [ ]:
# release the camera resource
cap.release()
# close window
cv.destroyAllWindows()
sock.close()